In [1]:
import pandas as pd
import numpy as np
import functools as ft
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
COLORS = [
    '#00B0F0',
    '#FF0000'
]

In [3]:
dfs = []
for x in range(2004, 2024):
    y = pd.read_csv(f'data/{x}.csv', sep='\t').dropna(thresh=2).reset_index(drop=True)[['Country', f'{x-1}', f'{x}', f'{x+1}']].rename({
        f'{x-1}': f'{x-1}_in_{x}',
        f'{x}': f'{x}_in_{x}',
        f'{x+1}': f'{x+1}_in_{x}',
    }, axis=1)
    y[f'{x-1}_in_{x}'] = y[f'{x-1}_in_{x}'].astype(str).str.replace(' ', '').str.replace('--', 'NaN').astype(float)
    y[f'{x}_in_{x}'] = y[f'{x}_in_{x}'].astype(str).str.replace(' ', '').str.replace('--', 'NaN').astype(float)
    y[f'{x+1}_in_{x}'] = y[f'{x+1}_in_{x}'].astype(str).str.replace(' ', '').str.replace('--', 'NaN').astype(float)
    dfs.append(y)
df = ft.reduce(lambda left, right: pd.merge(left, right, on='Country'), dfs)
df.dropna(inplace=True)
df.head()

,Country,2003_in_2004,2004_in_2004,2005_in_2004,2004_in_2005,2005_in_2005,2006_in_2005,2005_in_2006,2006_in_2006,2007_in_2006,...,2021_in_2020,2020_in_2021,2021_in_2021,2022_in_2021,2021_in_2022,2022_in_2022,2023_in_2022,2022_in_2023,2023_in_2023,2024_in_2023
0,Antigua and Barbuda,2.5,1.0,0.5,4.1,2.4,2.6,5.0,7.1,3.9,...,4.701,-20.031,0.965,7.019,5.270,6.021,5.629,8.459,5.592,5.417
1,Argentina,8.8,7.0,4.0,9.0,7.5,4.2,9.2,8.0,6.0,...,4.889,-9.905,7.500,2.465,10.398,4.037,2.024,4.956,-2.500,2.750
2,Barbados,2.2,3.0,2.5,4.4,3.1,3.3,3.9,4.2,4.9,...,7.400,-18.000,3.300,8.500,0.700,10.500,5.000,9.800,4.500,3.900
3,Belize,9.4,3.0,3.3,4.6,2.2,2.7,3.5,5.3,2.6,...,8.000,-14.038,8.500,5.400,16.342,3.502,1.998,12.726,4.007,2.998
4,Bolivia,2.5,3.8,4.5,3.6,3.9,2.5,4.1,4.1,3.9,...,5.600,-8.829,4.950,4.000,6.106,3.800,3.200,3.478,1.800,1.800


In [4]:
df[df['Country'] == 'Guyana'][['2020_in_2021', '2021_in_2022', '2022_in_2023']]

,2020_in_2021,2021_in_2022,2022_in_2023
15,43.48,23.774,62.288


In [5]:
# wtf?!
df = df[df['Country'] != 'Guyana']

In [6]:
for x in range(2004, 2023):
    df[f'expected_growth_{x}_t'] = df[f'{x}_in_{x}'].astype(float)
    df[f'expected_growth_{x}_t1'] = df[f'{x+1}_in_{x}'].astype(float)
    df[f'actual_growth_{x}'] = df[f'{x}_in_{x+1}'].astype(float)
df[['expected_growth_2008_t', 'expected_growth_2008_t1', 'actual_growth_2008']].head()

,expected_growth_2008_t,expected_growth_2008_t1,actual_growth_2008
0,2.092,4.009,2.800
1,6.500,3.600,6.758
2,1.700,1.000,0.180
3,4.000,2.500,3.771
4,5.900,5.000,6.100


In [7]:
base = {}
for x in range(2005, 2023):
    base[f'year_{x}'] = 0
for c in df['Country'].unique():
    base[f'country_{c}'] = 0

table = []
for x in range(2005, 2023):
    for c in df['Country'].unique():
        table.append({
            **base,
            'country': c,
            f'year_{x}': 1,
            f'country_{c}': 1,
            # 'expected_growth_t': df[f'expected_growth_{x}_t'][df['Country'] == c].iloc[0],
            'expected_growth_t1': df[f'expected_growth_{x}_t1'][df['Country'] == c].iloc[0] - df[f'expected_growth_{x}_t'][df['Country'] == c].iloc[0],
            'expected_growth_error': df[f'actual_growth_{x}'][df['Country'] == c].iloc[0] - df[f'expected_growth_{x}_t'][df['Country'] == c].iloc[0],
        })
X = pd.DataFrame(table).drop(['country_Mexico', 'year_2005'], axis=1)
groups = X['country']
X.drop('country', axis=1, inplace=True)
X.head()

,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,...,country_Peru,country_St. Kitts and Nevis,country_St. Lucia,country_St. Vincent and the Grenadines,country_Suriname,country_Trinidad and Tobago,country_Uruguay,country_Venezuela,expected_growth_t1,expected_growth_error
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.2,2.6
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-3.3,1.7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.2,0.8
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.5,1.3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1.4,0.2


In [8]:
y = X['expected_growth_error']
model = sm.OLS(y, sm.add_constant(X.drop(['expected_growth_error'], axis=1)))
fitted_model = model.fit(cov_type='cluster', cov_kwds={'groups': groups})
fitted_model.summary()

/home/seppo/.local/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 46, but rank is 18
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     expected_growth_error   R-squared:                       0.209
Model:                               OLS   Adj. R-squared:                  0.133
Method:                    Least Squares   F-statistic:                     29.15
Date:                   Thu, 20 Feb 2025   Prob (F-statistic):           1.60e-13
Time:                           22:16:49   Log-Likelihood:                -1039.8
No. Observations:                    522   AIC:                             2174.
Df Residuals:                        475   BIC:                             2374.
Df Model:                             46                                         
Covariance Type:                 cluster                                         
==========================================================================================================
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
const                                      0.7434      0.254      2.924      0.003       0.245       1.242
year_2006                                  0.1444      0.356      0.406      0.685      -0.552       0.841
year_2007                                 -0.4724      0.370     -1.276      0.202      -1.198       0.253
year_2008                                 -1.2842      0.325     -3.946      0.000      -1.922      -0.646
year_2009                                 -1.2543      0.476     -2.636      0.008      -2.187      -0.322
year_2010                                 -0.4104      0.425     -0.967      0.334      -1.243       0.422
year_2011                                 -1.2873      0.483     -2.665      0.008      -2.234      -0.341
year_2012                                 -0.9650      0.327     -2.948      0.003      -1.607      -0.323
year_2013                                 -0.6635      0.302     -2.195      0.028      -1.256      -0.071
year_2014                                 -0.6828      0.258     -2.644      0.008      -1.189      -0.177
year_2015                                 -1.0610      0.341     -3.109      0.002      -1.730      -0.392
year_2016                                 -1.3274      0.414     -3.205      0.001      -2.139      -0.516
year_2017                                 -1.2916      0.498     -2.592      0.010      -2.268      -0.315
year_2018                                 -0.4985      0.452     -1.103      0.270      -1.384       0.387
year_2019                                 -1.6196      0.336     -4.823      0.000      -2.278      -0.961
year_2020                                 -2.3687      1.311     -1.807      0.071      -4.938       0.200
year_2021                                  1.2053      0.673      1.790      0.073      -0.114       2.525
year_2022                                 -0.0014      0.523     -0.003      0.998      -1.027       1.024
country_Antigua and Barbuda                0.6493      0.074      8.801      0.000       0.505       0.794
country_Argentina                          1.0847      0.092     11.739      0.000       0.904       1.266
country_Barbados                          -0.9536      0.016    -58.518      0.000      -0.986      -0.922
country_Belize                             0.6248      0.014     44.186      0.000       0.597       0.653
country_Bolivia                            0.1818      0.058      3.109      0.002       0.067       0.296
country_Brazil                            -0.1085      0.039     -2.788      0.005      -0.185      -0.032
country_Chile                             -0.1607      0.082     -1.964      0.050      -0.321      -0.000
country_Colombia                           0.5094     

In [11]:
y.mean(), y.std()

(0.26620881226053644, 1.9962361936801627)